#RUN FROM HERE

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pandas as pd



# Correct paths (already correctly defined in your variables)
data_file_path = '/content/drive/MyDrive/GuidingRobot/SMALL_MAP_DATA/lidar_like_data_with_total_score.csv'
#labels_file_path = '/content/drive/MyDrive/GuidingRobot/LIDAR_DATA/LIDAR_DATA_UNZIPPED/game_and_map_data_lidar_like_with_walls_arranged/labels.csv'

# Load the datasets
df= pd.read_csv(data_file_path)
#df_labels = pd.read_csv(labels_file_path)
# Load data
# Assuming df_data and df_labels are your DataFrames

#df_labels = df_labels.drop(columns=['file_name'])



In [ ]:
df = df.dropna()


In [3]:
df = df.drop(columns=['file_name'])

In [4]:
print(df.columns)

Index(['beam_000_dist_to_obj0', 'beam_000_dist_to_obj1',
       'beam_000_dist_to_obj2', 'beam_001_dist_to_obj0',
       'beam_001_dist_to_obj1', 'beam_001_dist_to_obj2',
       'beam_002_dist_to_obj0', 'beam_002_dist_to_obj1',
       'beam_002_dist_to_obj2', 'beam_003_dist_to_obj0',
       ...
       'beam_358_dist_to_obj1', 'beam_358_dist_to_obj2',
       'beam_359_dist_to_obj0', 'beam_359_dist_to_obj1',
       'beam_359_dist_to_obj2', 'P1_goal_dist_x', 'P1_goal_dist_y',
       'final_row_flag', 'P1_action', 'Total_Score'],
      dtype='object', length=1085)


In [5]:
X = df.drop(['P1_action', 'Total_Score', 'final_row_flag'], axis=1)




In [ ]:


# Print unique values to verify the replacement
unique_values_after_replacement = df['P1_action'].unique()
print(unique_values_after_replacement)

In [6]:
y = df['P1_action']
rewards = df['Total_Score']

In [7]:
print("Dimensions of X:", X.shape)
print("Dimensions of y:", y.shape)
print("Dimensions of rewards:", rewards.shape)



Dimensions of X: (21806, 1082)
Dimensions of y: (21806,)
Dimensions of rewards: (21806,)


##split into train test val

 60% train, 20% validation, and 20% test split

In [8]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, X, y, rewards):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.long)
        self.rewards = torch.tensor(rewards.values, dtype=torch.float32)


        # Ensure the tensors are the correct shape for batch processing
        # If X was a DataFrame, it's now a 2D tensor, and we don't need to adjust its shape here

    def __len__(self):
        # Return the length of the dataset minus 1, so we have a next state for all but the last item
        return len(self.X) - 1

    def __getitem__(self, idx):
        # Return the current state, next state, and other information
        current_state = self.X[idx]
        next_state = self.X[idx + 1]  # Get the next state as the features at the next time step
        action = self.y[idx]
        reward = self.rewards[idx]


        return current_state, next_state, action, reward

# Assuming X, y, rewards, dones are defined elsewhere
# dataset = CustomDataset(X, y, rewards, dones)


In [9]:
import pandas as pd

# Assuming X is your original DataFrame with states

# Shift the DataFrame up to get the next states
next_states = X.shift(-1)

# Since the last row now contains NaN values (as there's no "next state" for the last entry),
# we drop the last row


# Now, next_states is a DataFrame where each row contains the "next state" from the original DataFrame


In [10]:
from sklearn.model_selection import train_test_split

# Split into train_val and test
X_train_val, X_test, y_train_val, y_test, rewards_train_val, rewards_test,  next_states_train_val, next_states_test = train_test_split(
    X, y, rewards, next_states, test_size=0.2, random_state=42, stratify=None)  # Remove stratify=None if your data or task doesn't support it

# Split train_val into train and validation
X_train, X_val, y_train, y_val, rewards_train, rewards_val,next_states_train, next_states_val = train_test_split(
    X_train_val, y_train_val, rewards_train_val,next_states_train_val, test_size=0.25, random_state=42, stratify=None)  # Adjust stratify as needed


##convert df to numpy array then tensor

In [11]:

from torch.utils.data import DataLoader, TensorDataset

# Convert your splits to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
rewards_train_tensor = torch.tensor(rewards_train.values, dtype=torch.float32)


X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.long)
rewards_val_tensor = torch.tensor(rewards_val.values, dtype=torch.float32)


X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)
rewards_test_tensor = torch.tensor(rewards_test.values, dtype=torch.float32)


# Assuming next_states_train, next_states_val, and next_states_test are your splits
next_states_train_tensor = torch.tensor(next_states_train.values, dtype=torch.float32)
next_states_val_tensor = torch.tensor(next_states_val.values, dtype=torch.float32)
next_states_test_tensor = torch.tensor(next_states_test.values, dtype=torch.float32)

# Create Te
# Extend the TensorDataset creation to include next_states
train_dataset = TensorDataset(X_train_tensor, y_train_tensor, rewards_train_tensor,  next_states_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor, rewards_val_tensor,  next_states_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor, rewards_test_tensor,  next_states_test_tensor)

# Create DataLoaders
batch_size = 64  # You can adjust the batch size

# Create DataLoader for each dataset
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



##Create Data Loader

##create model

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


import torch.nn as nn
import torch.nn.functional as F

class BCAgent(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BCAgent, self).__init__()
        # Use the input_dim and output_dim arguments to adjust layer dimensions
        self.fc1 = nn.Linear(input_dim, 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        self.dp1 = nn.Dropout(0.2)

        self.fc2 = nn.Linear(1024, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.dp2 = nn.Dropout(0.2)

        self.fc3 = nn.Linear(512, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.dp3 = nn.Dropout(0.2)

        self.fc4 = nn.Linear(128, 64)
        self.bn4 = nn.BatchNorm1d(64)
        self.dp4 = nn.Dropout(0.2)

        self.fc5 = nn.Linear(64, 32)
        self.bn5 = nn.BatchNorm1d(32)
        self.dp5 = nn.Dropout(0.2)

        self.fc6 = nn.Linear(32, output_dim)  # Use output_dim for the output layer

    def forward(self, x):
        x = F.relu(self.dp1(self.bn1(self.fc1(x))))
        x = F.relu(self.dp2(self.bn2(self.fc2(x))))
        x = F.relu(self.dp3(self.bn3(self.fc3(x))))
        x = F.relu(self.dp4(self.bn4(self.fc4(x))))
        x = F.relu(self.dp5(self.bn5(self.fc5(x))))
        x = self.fc6(x)
        return x

In [ ]:
''''
**OLD**

model = Sequential([
    Dense(1024, activation='relu', input_shape=(26,), ),
    Dense(512, activation='relu', ),
    Dense(128, activation='relu', ),
    Dense(64, activation='relu', ),
    Dense(32, activation='relu', ),
    # Update the Dense layer for output according to your classification/regression needs
    Dense(5, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
'''

In [ ]:
'''
model = Sequential([
    Dense(512, activation='relu', input_shape=(26,), ),

    Dense(128, activation='relu', ),

    Dense(32, activation='relu', ),
    # Update the Dense layer for output according to your classification/regression needs
    Dense(5, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
##RUN ON WAND B NAME-SOFT MEDOW
'''

In [ ]:

''''
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

# Assuming X_train.shape[1] is 126, replace it with 126 directly if you're not sure
model = Sequential([
    Dense(1024, activation='relu', input_shape=(26,), ),
    Dense(512, activation='relu', ),
    Dense(256, activation='relu', ),
    Dense(128, activation='relu', ),
    Dense(64, activation='relu', ),
    Dense(32, activation='relu', ),
    # Update the Dense layer for output according to your classification/regression needs
    Dense(5, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
###increasing depth deterioraes performance

##RUN ON WAND B NAME-GIDDY BUTTERFLY

''''

In [13]:
class ValueNetwork(nn.Module):
    def __init__(self, input_dim, dropout_rate=0.2):  # Added dropout_rate with a default value
        super(ValueNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.dropout1 = nn.Dropout(dropout_rate)  # Dropout layer after fc1
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(dropout_rate)  # Dropout layer after fc2
        self.fc3 = nn.Linear(64, 32)
        self.dropout3 = nn.Dropout(dropout_rate)  # Dropout layer after fc3
        self.fc4 = nn.Linear(32, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)  # Apply dropout after activation
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)  # Apply dropout after activation
        x = F.relu(self.fc3(x))
        x = self.dropout3(x)  # Apply dropout after activation
        x = self.fc4(x)  # No dropout before the final layer
        return x



##train

In [14]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.9/264.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00


In [15]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

In [16]:
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint, WandbEvalCallback


wandb.login()



<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [17]:
import wandb
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import torch

# EarlyStopping class to stop the training when a monitored metric has stopped improving.
class EarlyStopping:
    def __init__(self, patience=7, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = np.Inf
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                print('Early stopping')
                self.early_stop = True


In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import wandb

# Assuming BCAgent and ValueNetwork classes are defined above


# Initialize Weights & Biases with separate learning rates for each model
wandb.init(project='GRProject weighted bc final small map 01', config={
    "learning_rate_bc": 0.0000001,  # Learning rate for the BCAgent model
    "learning_rate_value": 0.0000001,  # Learning rate for the ValueNetwork model
    "epochs": 20,
    "batch_size": 64,
    "gamma": 0.009,  # Discount factor for advantage calculation
    "alpha": 0.005,
    # Add any other hyperparameters you might have
})
config = wandb.config


input_dim = X.shape[1]  # Number of columns in the DataFrame

bc_model = BCAgent(input_dim=input_dim, output_dim=5)
value_model = ValueNetwork(input_dim=input_dim)

# Define optimizers and loss function for BC model
bc_optimizer = optim.Adam(bc_model.parameters(), lr=config.learning_rate_bc)
bc_criterion = nn.CrossEntropyLoss()  # Assuming a classification task
value_criterion = nn.MSELoss()
value_optimizer = optim.Adam(value_model.parameters(), lr=config.learning_rate_value)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bc_model.to(device)
value_model.to(device)
# Create an iterator from the train_loader
data_iterator = iter(train_loader)
# Fetch the first batch
current_batch = next(data_iterator)
data_iterator_v = iter(val_loader)
# Fetch the first batch
current_batch_v = next(data_iterator_v)
# Assuming EarlyStopping class and config are defined
early_stopper = EarlyStopping(patience=10, min_delta=0.001)


Epoch,▁▂▃▃▄▅▆▆▇█
Train Accuracy,█▃▂▁▂▁▁▁▁▁
Train Loss,█▆▅▅▄▃▃▂▂▁
Validation Accuracy,██▁▁▁▁▁▁▁▁
Validation Loss,█▇▆▅▄▄▃▂▂▁
Epoch,10
Train Accuracy,0.0131
Train Loss,-4.12463
Validation Accuracy,0.0
Validation Loss,-4.1503


In [43]:

for epoch in range(config.epochs):
    bc_model.train()
    value_model.train()  # If your value model also needs training updates
    train_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    for i in range(len(train_loader) -1):  # We do -1 because we've already fetched the first batch
        # Unpack the current batch

        inputs, actions, rewards,next_states = current_batch # Current batch (ignoring dones for now)
        inputs, actions, rewards,next_states = inputs.to(device), actions.to(device), rewards.to(device),next_states.to(device)
        # Behavior cloning model predictions and loss
        bc_logits = bc_model(inputs)
         # Convert logits to log probabilities
        log_probs = F.log_softmax(bc_logits, dim=1)

        # Gather the log probabilities for the taken actions
        gathered_log_probs = torch.gather(log_probs, 1, actions.unsqueeze(-1)).squeeze(-1)


        bc_loss = bc_criterion(bc_logits, actions)


        #Value network predictions
        state_values = value_model(inputs)
        next_state_values=value_model(next_states)
        actions = actions.squeeze()
        loss = value_criterion(state_values, actions)

        # Print shapes to debug
            # Compute advantage
        advantages = rewards + (config.gamma * next_state_values) - state_values

        # Calculate the exponential of the scaled advantage
        exp_scaled_advantages = torch.exp(config.alpha * advantages)

        # Calculate the weighted behavior cloning loss
        weighted_bc_loss = (gathered_log_probs * exp_scaled_advantages.detach()).mean()

        # Update BC model
        bc_optimizer.zero_grad()
        weighted_bc_loss.backward()
        torch.nn.utils.clip_grad_norm_(bc_model.parameters(), max_norm=1.0)
        bc_optimizer.step()

        train_loss += weighted_bc_loss.item() * inputs.size(0)

        _, predicted = torch.max(bc_logits.data, 1)
        correct_predictions += (predicted == actions).sum().item()
        total_predictions += config.batch_size





    if total_predictions != 0:
        train_loss /= len(train_loader.dataset)

        train_accuracy = (correct_predictions / total_predictions)
    else:
        train_loss = 0.0
        train_accuracy = 0.0


    # Validation Phase
    bc_model.eval()
    value_model.eval()  # Assuming value model evaluation is necessary
    val_loss = 0.0
    correct_val_predictions = 0
    total_val_predictions = 0



    with torch.no_grad():
      for i in range(len(val_loader) -1):  # We do -1 because we've already fetched the first batch
        # Unpack the current batch

            inputs, actions, rewards,next_states = current_batch # Current batch (ignoring dones for now)
            inputs, actions, rewards,next_states = inputs.to(device), actions.to(device), rewards.to(device),next_states.to(device)




            # Behavior cloning model predictions and loss
            bc_logits = bc_model(inputs)
            bc_loss = bc_criterion(bc_logits, actions)
            log_probs = F.log_softmax(bc_logits, dim=1)
            gathered_log_probs = torch.gather(log_probs, 1, actions.unsqueeze(-1)).squeeze(-1)


            state_values = value_model(inputs)
            next_state_values=value_model(next_states)
            actions = actions.squeeze()
            loss = value_criterion(state_values, actions)

            # Compute advantage
            advantages = rewards + config.gamma * next_state_values  - state_values

            # Calculate the exponential of the scaled advantage
            exp_scaled_advantages = torch.exp(config.alpha * advantages)

            # Calculate the weighted behavior cloning loss
            weighted_bc_loss = (gathered_log_probs * exp_scaled_advantages.detach()).mean()

            val_loss += weighted_bc_loss.item() * inputs.size(0)

            _, predicted = torch.max(bc_logits.data, 1)

            total_val_predictions += config.batch_size
            correct_val_predictions += (predicted == actions).sum().item()
      if total_val_predictions != 0:
              val_loss /= len(val_loader.dataset)
              val_accuracy = (correct_val_predictions / total_val_predictions)
      else:
              val_loss = 0.0
              val_accuracy = 0.0



    # Log metrics to WandB
    wandb.log({
        "Epoch": epoch + 1,
        "Train Loss": train_loss,
        "Train Accuracy": train_accuracy,
        "Validation Loss": val_loss,
        "Validation Accuracy": val_accuracy
    })

    print(f'Epoch {epoch+1}/{config.epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Early Stopping
    early_stopper(val_loss)
    if early_stopper.early_stop:
        print("Early stopping triggered.")
        break

# Close the WandB run
wandb.finish()



Epoch 1/20, Train Loss: -3.9675, Train Accuracy: 0.20%, Val Loss: -3.8489, Val Accuracy: 0.19%
Epoch 2/20, Train Loss: -3.9941, Train Accuracy: 0.20%, Val Loss: -3.9036, Val Accuracy: 0.19%
Epoch 3/20, Train Loss: -4.0225, Train Accuracy: 0.20%, Val Loss: -3.9203, Val Accuracy: 0.17%
Epoch 4/20, Train Loss: -4.0408, Train Accuracy: 0.19%, Val Loss: -3.9512, Val Accuracy: 0.17%


KeyboardInterrupt: 

##Inference

In [ ]:
bc_model.eval()  # Ensure the model is in evaluation mode
value_model.eval()  # If your value model is used during testing
test_loss = 0.0
correct_test_predictions = 0
total_test_predictions = 0

with torch.no_grad():  # No gradient computation for inference
    for inputs, actions, rewards, dones, next_states in test_loader:  # Iterate over the test_loader
        # Transfer data to the appropriate device
        inputs, actions, rewards, dones, next_states = inputs.to(device), actions.to(device), rewards.to(device), dones.to(device), next_states.to(device)

        # Behavior cloning model predictions and loss
        bc_logits = bc_model(inputs)
        bc_loss = bc_criterion(bc_logits, actions)

        # Value network predictions, if used in your testing logic
        state_values = value_model(inputs)
        next_state_values = value_model(next_states)
        actions = actions.squeeze()  # Ensure actions are correctly shaped for comparison
        loss = value_criterion(state_values, actions)  # Assuming you evaluate some loss for the value model

        # Compute advantage, if applicable for your testing logic
        advantages = rewards + (config.gamma * next_state_values)- state_values

        # Calculate the exponential of the scaled advantage
        exp_scaled_advantages = torch.exp(config.alpha * advantages)

        # Calculate the weighted behavior cloning loss, if applicable to test evaluation
        weighted_bc_loss = (bc_loss * exp_scaled_advantages.detach()).mean()

        test_loss += weighted_bc_loss.item() * inputs.size(0)  # Accumulate the batch loss

        _, predicted = torch.max(bc_logits.data, 1)
        total_test_predictions += actions.size(0)
        correct_test_predictions += (predicted == actions).sum().item()

# Calculate final metrics for the test set
if total_test_predictions != 0:
    test_loss /= len(test_loader.dataset)  # Average loss across all test dataset samples
    test_accuracy = (correct_test_predictions / total_test_predictions) * 100  # Percentage accuracy
else:
    test_loss = 0.0
    test_accuracy = 0.0

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")




ValueError: not enough values to unpack (expected 5, got 4)

In [ ]:


wandb.init()
torch.save(bc_model.state_dict(), '/content/drive/MyDrive/GuidingRobot/model_checkpoints_relative/bc_model_1.pth')
torch.save(value_model.state_dict(),  '/content/drive/MyDrive/GuidingRobot/model_checkpoints_relative/value_model_1.pth')
wandb.save( '/content/drive/MyDrive/GuidingRobot/model_checkpoints_relative/bc_model_1.pth')
wandb.save( '/content/drive/MyDrive/GuidingRobot/model_checkpoints_relative/value_model_1.pth')

####yet to do hyperparameter opt in pytorch

In [ ]:
import random
import numpy as np
import tensorflow as tf
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

# Simple Keras Model

# Launch 5 experiments, trying different dropout rates
for run in range(5):
    # Start a run, tracking hyperparameters
    wandb.init(project='Guiding robot project 8',
                 config={  # and include hyperparameters and metadata
                     "learning_rate": 0.010,
                     "epochs": 10,
                     "batch_size": 32,
                     "loss_function": "sparse_categorical_crossentropy",
                     "dropout": random.uniform(0.01, 0.80),

                 })
    config = wandb.config  # We'll use this to configure our experiment


    wandb_callbacks = [
        WandbMetricsLogger(),
        WandbModelCheckpoint(filepath="my_model_{epoch:02d}"),
    ]

    model.fit(
        x=X_train,
        y=y_train,
        epochs=config.epochs,

        validation_data=(X_val, y_val),
        test_data=(X_test, y_test),
        callbacks=wandb_callbacks,
    )
    # This simple block simulates a training loop logging metrics
    wandb.log({
        "Epoch": epoch,
        "Train Loss": loss_train,
        "Train Acc": acc_train,
        "Valid Loss": loss_valid,
        "Valid Acc": acc_valid})

    # Mark the run as finished
    wandb.finish()

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Problem at: <ipython-input-24-c49a9f7ca02e> 11 <cell line: 9>


CommError: Run initialization has timed out after 90.0 sec. 
Please refer to the documentation for additional information: https://docs.wandb.ai/guides/track/tracking-faq#initstarterror-error-communicating-with-wandb-process-

##Hypermarater sweeps

In [ ]:
import tensorflow as tf
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

In [ ]:
sweep_config = {
    "name": "sweepdemo",
    "method": "grid",
    "metric": {"goal": "minimize", "name": "validation_loss"},
    "parameters": {
        "learning_rate": {"values":[0.0001, 0.001,0.01, 0.1]},
        "batch_size": {"values": [16, 32, 64,128,256]},
        "epochs": {"values": [5, 10, 15]},
        "optimizer": {"values": ["adam", "sgd"]},
    },
}

In [ ]:
metric = {
    'name': 'loss',
    'goal': 'minimize'
    }

sweep_config['metric'] = metric

In [ ]:
parameters_dict = {
    'optimizer': {
        'values': ['adam']
        }

    }

sweep_config['parameters'] = parameters_dict
sweep_configuration = {

    },
}

In [ ]:
parameters_dict.update({
    'epochs': {
        'value': 10}
    })

In [ ]:

import math

parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0.001,
        'max': 0.1
      },
    'batch_size': {
        'values': [32, 64,128]
      }
    })


In [ ]:
import pprint

pprint.pprint(sweep_config)

{'method': 'grid',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'method': 'grid',
                'metric': {'goal': 'minimize', 'name': 'validation_loss'},
                'name': 'sweepdemo-GR',
                'parameters': {'batch_size': {'values': [16, 32, 64]},
                               'epochs': {'values': [5, 10, 15]},
                               'learning_rate': {'max': 0.1, 'min': 0.0001},
                               'optimizer': {'values': ['adam', 'sgd']}}}}


In [ ]:

def get_optimizer(lr=1e-3, optimizer="adam"):
    "Select optmizer between adam and sgd with momentum"
    if optimizer.lower() == "adam":
        return tf.keras.optimizers.Adam(learning_rate=lr)
    if optimizer.lower() == "sgd":
        return tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.1)

def train(model, batch_size=64, epochs=10, lr=1e-3, optimizer='adam', log_freq=10):

    # Compile model like you usually do.
    tf.keras.backend.clear_session()
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=get_optimizer(lr, optimizer),
                  metrics=["accuracy"])

    # callback setup
    wandb_callbacks = [
        WandbMetricsLogger(log_freq=log_freq),
        WandbModelCheckpoint(filepath="/content/drive/MyDrive/GuidingRobot/model_checkpoints_relative/model_checkpoint11.h5")
    ]

    model.fit(X_train,
              y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              callbacks=wandb_callbacks)

In [ ]:

def sweep_train(config_defaults=None):
    # Initialize wandb with a sample project name
    with wandb.init(config=config_defaults):  # this gets over-written in the Sweep


        train(model,
              wandb.config.batch_size,
              wandb.config.epochs,
              wandb.config.learning_rate,
              wandb.config.optimizer)

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="Guiding robot project 11")

Create sweep with ID: ikxdlv3t
Sweep URL: https://wandb.ai/hb12/Guiding%20robot%20project%2011/sweeps/ikxdlv3t


In [ ]:

wandb.agent(sweep_id, function=sweep_train, count=10)

wandb: Agent Starting Run: qsk2jmhn with config:
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	learning_rate: 0.001
wandb: 	optimizer: sgd


Epoch 1/5
2027/2031 [============================>.] - ETA: 0s - loss: 0.9502 - accuracy: 0.7230

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2031/2031 [==============================] - 19s 9ms/step - loss: 0.9498 - accuracy: 0.7231 - val_loss: 0.6413 - val_accuracy: 0.7786
Epoch 2/5
2031/2031 [==============================] - 19s 9ms/step - loss: 0.5429 - accuracy: 0.8107 - val_loss: 0.5545 - val_accuracy: 0.8014
Epoch 3/5
2031/2031 [==============================] - 19s 9ms/step - loss: 0.4777 - accuracy: 0.8265 - val_loss: 0.5046 - val_accuracy: 0.8143
Epoch 4/5
2031/2031 [==============================] - 21s 10ms/step - loss: 0.4413 - accuracy: 0.8366 - val_loss: 0.4719 - val_accuracy: 0.8276
Epoch 5/5
2031/2031 [==============================] - 19s 9ms/step - loss: 0.4122 - accuracy: 0.8440 - val_loss: 0.4326 - val_accuracy: 0.8373


batch/accuracy,▁▃▄▅▅▆▆▆▇▇▇▇▇▇▇▇████████████████████████
batch/batch_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,▁▆▇██
epoch/epoch,▁▃▅▆█
epoch/learning_rate,▁▁▁▁▁
epoch/loss,█▃▂▁▁
epoch/val_accuracy,▁▄▅▇█
epoch/val_loss,█▅▃▂▁
batch/accuracy,0.84399


...

wandb: Agent Starting Run: 5rdxce9b with config:
wandb: 	batch_size: 16
wandb: 	epochs: 10
wandb: 	learning_rate: 0.01
wandb: 	optimizer: adam


Epoch 1/10
2031/2031 [==============================] - ETA: 0s - loss: 1.5255 - accuracy: 0.2627

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2031/2031 [==============================] - 26s 12ms/step - loss: 1.5255 - accuracy: 0.2627 - val_loss: 1.5271 - val_accuracy: 0.2672
Epoch 2/10
2031/2031 [==============================] - 24s 12ms/step - loss: 1.5255 - accuracy: 0.2617 - val_loss: 1.5254 - val_accuracy: 0.2672
Epoch 3/10
2031/2031 [==============================] - 26s 13ms/step - loss: 1.5254 - accuracy: 0.2637 - val_loss: 1.5261 - val_accuracy: 0.2672
Epoch 4/10
2031/2031 [==============================] - 26s 13ms/step - loss: 1.5255 - accuracy: 0.2620 - val_loss: 1.5267 - val_accuracy: 0.2672
Epoch 5/10
2031/2031 [==============================] - 25s 12ms/step - loss: 1.5256 - accuracy: 0.2627 - val_loss: 1.5254 - val_accuracy: 0.2520
Epoch 6/10
2031/2031 [==============================] - 24s 12ms/step - loss: 1.5254 - accuracy: 0.2610 - val_loss: 1.5254 - val_accuracy: 0.2672
Epoch 7/10
2031/2031 [==============================] - 25s 12ms/step - loss: 1.5256 - accuracy: 0.2626 - val_loss: 1.5249 - val_accura

batch/accuracy,▇▆▅▅▁▅▇▆█▆▇▆▂▇▇▆█▇▆▆██▆▅▄▆▇▆▅▆▅▆▅▆▇▆▇▇▇▆
batch/batch_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,▄▃▃▃█▃▂▃▂▃▃▃▃▂▃▃▁▂▂▃▂▃▃▃▃▂▃▃▂▃▃▃▂▃▂▃▃▃▃▃
epoch/accuracy,▅▃▇▃▅▁▄█▄▂
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,▃▄▂▄▆▂█▁▄▅
epoch/val_accuracy,████▁█████
epoch/val_loss,▆▂▄▅▂▂▁█▇▂
batch/accuracy,0.2613


In [ ]:
run.finish()

NameError: name 'run' is not defined

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
wandb.log({'test_loss': test_loss, 'test_accuracy': test_accuracy})


339/339 [==============================] - 1s 4ms/step - loss: 1.5251 - accuracy: 0.2672


Error: You must call wandb.init() before wandb.log()

In [ ]:
run.finish()

epoch/acc,▁▇▅█▃▄▇█▂▅
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▃▂▃▄▂▂▁▄▃
epoch/val_acc,▆▇▇▁█▅▅█▄▅
epoch/val_loss,▃▁▃█▁▄▄▁▄▄
test_accuracy,▁█
test_loss,█▁
epoch/acc,0.73959
epoch/epoch,9
epoch/learning_rate,0.01


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print('Test Error Rate: ', round((1 - accuracy) * 100, 2))

# With wandb.log, we can easily pass in metrics as key-value pairs.
wandb.log({'Test Error Rate': round((1 - accuracy) * 100, 2)})



339/339 [==============================] - 1s 3ms/step - loss: 1.5251 - accuracy: 0.2672
Test Error Rate:  73.28


Error: You must call wandb.init() before wandb.log()

##test

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}, Test loss: {test_loss}")


339/339 [==============================] - 1s 3ms/step - loss: 1.5251 - accuracy: 0.2672
Test accuracy: 0.26719599962234497, Test loss: 1.5250768661499023


In [ ]:
predictions = model.predict(X_test)
predicted_classes = predictions.argmax(axis=1)  # Assuming a classification problem



In [ ]:
# Creating a DataFrame with true labels and predicted labels
results_df = pd.DataFrame({'True_Label': y_test, 'Predicted_Label': predicted_classes})


In [ ]:
# Display the first few rows of the DataFrame to see overall predictions
print(results_df.head())

# Optional: Display where predictions were correct
correct_predictions_df = results_df[results_df['True_Label'] == results_df['Predicted_Label']]
print("\nCorrect Predictions:")
print(correct_predictions_df.head())

# Optional: Display where predictions were incorrect
incorrect_predictions_df = results_df[results_df['True_Label'] != results_df['Predicted_Label']]
print("\nIncorrect Predictions:")
print(incorrect_predictions_df.head())
